# Legal Clause Risk Scorer - Data Exploration and Model Development

This notebook provides comprehensive exploration of legal contract data and demonstrates the capabilities of our risk assessment model.

## Table of Contents
1. [Setup and Configuration](#setup)
2. [Data Loading and Exploration](#data-exploration)
3. [Text Preprocessing Analysis](#preprocessing)
4. [Model Architecture Overview](#model-architecture)
5. [Training Demonstration](#training)
6. [Risk Assessment Examples](#risk-examples)
7. [Performance Analysis](#performance)
8. [Interactive Risk Scoring](#interactive)

## 1. Setup and Configuration {#setup}

In [ ]:
import sys
from pathlib import Path

# Add src to path for imports
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / "src"))

# Core imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.auto import tqdm

# Legal risk scorer imports
from legal_clause_risk_scorer.utils.config import load_config, set_random_seeds
from legal_clause_risk_scorer.data.loader import LegalDataLoader
from legal_clause_risk_scorer.data.preprocessing import LegalTextPreprocessor
from legal_clause_risk_scorer.models.model import LegalClauseRiskModel
from legal_clause_risk_scorer.evaluation.metrics import LegalRiskMetrics

# Configure display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', 100)
plt.style.use('seaborn-v0_8')
warnings.filterwarnings('ignore')

print("✅ Environment setup complete!")

In [ ]:
# Load configuration
config = load_config()
set_random_seeds(config)

print("📋 Configuration loaded:")
print(f"  Base model: {config.get('model.base_model')}")
print(f"  Batch size: {config.get('training.batch_size')}")
print(f"  Learning rate: {config.get('training.learning_rate')}")
print(f"  Target F1: {config.get('evaluation.targets.clause_detection_f1')}")
print(f"  Target MAE: {config.get('evaluation.targets.risk_score_mae')}")

## 2. Data Loading and Exploration {#data-exploration}

In [ ]:
# Initialize data loader
data_loader = LegalDataLoader(config)
print("📁 Data loader initialized")

# Load sample data for exploration (smaller subset for notebook)
try:
    # Load a small sample of CUAD data first
    print("Loading sample legal contract data...")
    cuad_sample = data_loader.load_cuad_dataset()
    print(f"✅ Loaded {len(cuad_sample)} CUAD samples")
    
    # Convert to pandas for exploration
    cuad_df = cuad_sample.to_pandas()
    
except Exception as e:
    print(f"⚠️ Could not load CUAD data: {e}")
    print("Creating synthetic sample data for demonstration...")
    
    # Create synthetic legal clause data for demonstration
    synthetic_clauses = [
        {
            'text': "The Employee may terminate this agreement at any time with 30 days written notice.",
            'clause_type': 'termination',
            'risk_score': 3.2,
            'risk_category': 'low_risk'
        },
        {
            'text': "The Company may terminate Employee's employment at any time, with or without cause, and with or without notice.",
            'clause_type': 'termination',
            'risk_score': 8.7,
            'risk_category': 'high_risk'
        },
        {
            'text': "Employee agrees not to compete with the Company for a period of 2 years following termination.",
            'clause_type': 'non_compete',
            'risk_score': 7.5,
            'risk_category': 'high_risk'
        },
        {
            'text': "All intellectual property created during employment shall belong to the Company.",
            'clause_type': 'intellectual_property',
            'risk_score': 6.2,
            'risk_category': 'medium_risk'
        },
        {
            'text': "Employee shall receive base salary plus performance bonuses as determined by management.",
            'clause_type': 'compensation',
            'risk_score': 4.8,
            'risk_category': 'medium_risk'
        }
    ]
    
    cuad_df = pd.DataFrame(synthetic_clauses)
    print(f"✅ Created {len(cuad_df)} synthetic samples")

# Display basic information
print("\n📊 Dataset Overview:")
print(cuad_df.info())
cuad_df.head()

In [ ]:
# Analyze risk score distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Risk score histogram
axes[0].hist(cuad_df['risk_score'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
axes[0].set_title('Risk Score Distribution')
axes[0].set_xlabel('Risk Score (1-10)')
axes[0].set_ylabel('Frequency')

# Risk category distribution
risk_counts = cuad_df['risk_category'].value_counts()
axes[1].pie(risk_counts.values, labels=risk_counts.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Risk Category Distribution')

# Clause type distribution
clause_counts = cuad_df['clause_type'].value_counts()
axes[2].bar(clause_counts.index, clause_counts.values, color='lightcoral')
axes[2].set_title('Clause Type Distribution')
axes[2].set_xlabel('Clause Type')
axes[2].set_ylabel('Count')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n📈 Risk Score Statistics:")
print(cuad_df['risk_score'].describe())

print("\n🏷️ Risk Category Counts:")
print(cuad_df['risk_category'].value_counts())

In [ ]:
# Analyze text characteristics
cuad_df['text_length'] = cuad_df['text'].str.len()
cuad_df['word_count'] = cuad_df['text'].str.split().str.len()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Text length distribution
axes[0].hist(cuad_df['text_length'], bins=20, alpha=0.7, color='lightgreen')
axes[0].set_title('Text Length Distribution')
axes[0].set_xlabel('Characters')
axes[0].set_ylabel('Frequency')

# Word count distribution
axes[1].hist(cuad_df['word_count'], bins=20, alpha=0.7, color='orange')
axes[1].set_title('Word Count Distribution')
axes[1].set_xlabel('Words')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

print("\n📏 Text Statistics:")
print(f"Average text length: {cuad_df['text_length'].mean():.1f} characters")
print(f"Average word count: {cuad_df['word_count'].mean():.1f} words")
print(f"Longest text: {cuad_df['text_length'].max()} characters")
print(f"Shortest text: {cuad_df['text_length'].min()} characters")

## 3. Text Preprocessing Analysis {#preprocessing}

In [ ]:
# Initialize text preprocessor
preprocessor = LegalTextPreprocessor(config)
print("🔧 Text preprocessor initialized")

# Demonstrate preprocessing on sample texts
sample_texts = [
    "The Employee shall not, during the term of employment and for two (2) years thereafter, directly or indirectly compete with the Company.",
    "This Agreement shall terminate upon 30 days written notice by either party, provided that such termination shall not affect any accrued obligations.",
    "All intellectual property, including but not limited to patents, copyrights, and trade secrets, developed during employment shall be the exclusive property of Company."
]

print("\n🧹 Preprocessing Examples:")
for i, text in enumerate(sample_texts, 1):
    cleaned = preprocessor.clean_legal_text(text)
    print(f"\n--- Example {i} ---")
    print(f"Original: {text}")
    print(f"Cleaned:  {cleaned}")
    print(f"Length change: {len(text)} → {len(cleaned)} characters")

In [ ]:
# Extract features for analysis
if len(cuad_df) > 0:
    sample_texts_for_features = cuad_df['text'].tolist()[:10]  # Use first 10 samples
    
    print("🔍 Extracting text features...")
    features = preprocessor.extract_features(sample_texts_for_features)
    
    # Create features DataFrame
    features_df = pd.DataFrame({
        'text_length': features['text_length'],
        'word_count': features['word_count'],
        'sentence_count': features['sentence_count'],
        'legal_terms_count': features['legal_terms_count'],
        'risk_keywords_count': features['risk_keywords_count'],
        'modal_verbs_count': features['modal_verbs_count'],
        'avg_sentence_length': features['avg_sentence_length'],
        'avg_word_length': features['avg_word_length']
    })
    
    print("\n📊 Feature Statistics:")
    print(features_df.describe())
    
    # Visualize feature correlations
    plt.figure(figsize=(10, 8))
    correlation_matrix = features_df.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.2f')
    plt.title('Feature Correlation Matrix')
    plt.tight_layout()
    plt.show()
else:
    print("⚠️ No data available for feature extraction")

## 4. Model Architecture Overview {#model-architecture}

In [ ]:
# Initialize model for architecture exploration
print("🏗️ Initializing model architecture...")

try:
    model = LegalClauseRiskModel(config)
    print("✅ Model initialized successfully")
    
    # Print model architecture
    print("\n🔍 Model Architecture:")
    print(model)
    
    # Count parameters
    from legal_clause_risk_scorer.models.model import count_parameters
    param_counts = count_parameters(model)
    
    print("\n📊 Model Parameters:")
    print(f"  Total parameters: {param_counts['total_parameters']:,}")
    print(f"  Trainable parameters: {param_counts['trainable_parameters']:,}")
    print(f"  Frozen parameters: {param_counts['frozen_parameters']:,}")
    
    # Model configuration details
    print("\n⚙️ Model Configuration:")
    print(f"  Base model: {config.get('model.base_model')}")
    print(f"  Hidden size: {config.get('model.hidden_size')}")
    print(f"  Number of labels: {config.get('model.num_labels')}")
    print(f"  Dropout rate: {config.get('model.dropout')}")
    
except Exception as e:
    print(f"⚠️ Could not initialize full model: {e}")
    print("This may be due to missing model weights or dependencies.")

## 5. Training Demonstration {#training}

In [ ]:
# Demonstrate training pipeline setup
print("🚂 Training Pipeline Demonstration")
print("(Note: This is a setup demonstration, not actual training)")

# Show training configuration
training_config = config.get_section('training')
print("\n📋 Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

# Demonstrate data loading for training
print("\n📊 Data Loading for Training:")
print("1. Load CUAD and LEDGAR datasets")
print("2. Combine and balance datasets")
print("3. Create train/validation/test splits")
print("4. Tokenize text data")
print("5. Create PyTorch DataLoaders")

# Show evaluation configuration
evaluation_config = config.get_section('evaluation')
print("\n🎯 Evaluation Configuration:")
print(f"  Metrics: {evaluation_config['metrics']}")
print(f"  Targets: {evaluation_config['targets']}")

# MLflow configuration
mlflow_config = config.get_section('mlflow')
print("\n📈 Experiment Tracking:")
for key, value in mlflow_config.items():
    print(f"  {key}: {value}")

## 6. Risk Assessment Examples {#risk-examples}

In [ ]:
# Define example legal clauses with varying risk levels
example_clauses = [
    {
        'text': "Employee may terminate this agreement with 30 days written notice.",
        'expected_risk': 'low',
        'explanation': "Gives employee reasonable termination rights"
    },
    {
        'text': "Company may terminate employee immediately without cause or notice.",
        'expected_risk': 'high',
        'explanation': "At-will termination with no protection for employee"
    },
    {
        'text': "Employee agrees to non-disclosure of proprietary information during and after employment.",
        'expected_risk': 'medium',
        'explanation': "Standard confidentiality clause"
    },
    {
        'text': "Employee shall not compete with company for 5 years in any capacity worldwide.",
        'expected_risk': 'high',
        'explanation': "Overly broad non-compete restriction"
    },
    {
        'text': "Salary will be reviewed annually with potential for merit-based increases.",
        'expected_risk': 'low',
        'explanation': "Fair compensation review process"
    }
]

# Initialize risk assessment components
print("⚖️ Legal Clause Risk Assessment Examples")
print("\nAnalyzing example clauses for risk indicators...")

# Manual risk scoring based on keywords and patterns
def assess_clause_risk(text):
    """Simple rule-based risk assessment for demonstration."""
    text_lower = text.lower()
    
    high_risk_indicators = [
        'without cause', 'without notice', 'immediately', 'at will',
        'sole discretion', 'worldwide', 'unlimited', 'perpetual',
        'waive', 'forfeit', 'irrevocable'
    ]
    
    protective_indicators = [
        'with notice', 'reasonable', 'fair', 'written notice',
        'mutual', 'subject to', 'limited', 'annual review'
    ]
    
    risk_score = 5.0  # Neutral starting point
    
    for indicator in high_risk_indicators:
        if indicator in text_lower:
            risk_score += 1.5
    
    for indicator in protective_indicators:
        if indicator in text_lower:
            risk_score -= 1.0
    
    # Clamp to 1-10 range
    return max(1.0, min(10.0, risk_score))

# Assess each example clause
results_df = []
for i, clause in enumerate(example_clauses, 1):
    risk_score = assess_clause_risk(clause['text'])
    
    if risk_score <= 3.5:
        risk_category = 'low_risk'
        color = '🟢'
    elif risk_score <= 7.0:
        risk_category = 'medium_risk'
        color = '🟡'
    else:
        risk_category = 'high_risk'
        color = '🔴'
    
    print(f"\n--- Example {i} {color} ---")
    print(f"Text: {clause['text']}")
    print(f"Risk Score: {risk_score:.1f}/10")
    print(f"Risk Category: {risk_category}")
    print(f"Expected: {clause['expected_risk']} risk")
    print(f"Explanation: {clause['explanation']}")
    
    results_df.append({
        'clause': clause['text'][:50] + '...' if len(clause['text']) > 50 else clause['text'],
        'risk_score': risk_score,
        'risk_category': risk_category,
        'expected': clause['expected_risk']
    })

# Create summary DataFrame
results_df = pd.DataFrame(results_df)
print("\n📊 Risk Assessment Summary:")
print(results_df)

In [ ]:
# Visualize risk assessment results
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Risk score comparison
clause_nums = range(1, len(results_df) + 1)
colors = ['green' if cat == 'low_risk' else 'orange' if cat == 'medium_risk' else 'red' 
          for cat in results_df['risk_category']]

axes[0].bar(clause_nums, results_df['risk_score'], color=colors, alpha=0.7)
axes[0].set_title('Risk Scores by Clause')
axes[0].set_xlabel('Clause Number')
axes[0].set_ylabel('Risk Score (1-10)')
axes[0].set_ylim(0, 10)

# Add horizontal lines for risk thresholds
axes[0].axhline(y=3.5, color='orange', linestyle='--', alpha=0.5, label='Low/Medium threshold')
axes[0].axhline(y=7.0, color='red', linestyle='--', alpha=0.5, label='Medium/High threshold')
axes[0].legend()

# Risk category distribution
category_counts = results_df['risk_category'].value_counts()
axes[1].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%',
           colors=['green', 'orange', 'red'])
axes[1].set_title('Risk Category Distribution')

plt.tight_layout()
plt.show()

print("\n🎯 Assessment Accuracy:")
expected_map = {'low': 'low_risk', 'medium': 'medium_risk', 'high': 'high_risk'}
results_df['expected_category'] = results_df['expected'].map(expected_map)
accuracy = (results_df['risk_category'] == results_df['expected_category']).mean()
print(f"Category prediction accuracy: {accuracy:.1%}")

## 7. Performance Analysis {#performance}

In [ ]:
# Initialize metrics evaluator
metrics_evaluator = LegalRiskMetrics(config)
print("📊 Performance Analysis")

# Simulate evaluation results for demonstration
np.random.seed(42)
n_samples = 100

# Generate synthetic evaluation data
y_true_class = np.random.choice([0, 1, 2], size=n_samples, p=[0.4, 0.35, 0.25])
y_pred_class = y_true_class + np.random.choice([-1, 0, 1], size=n_samples, p=[0.1, 0.8, 0.1])
y_pred_class = np.clip(y_pred_class, 0, 2)

y_true_score = np.random.normal(5.0, 2.0, n_samples)
y_true_score = np.clip(y_true_score, 1, 10)
y_pred_score = y_true_score + np.random.normal(0, 0.8, n_samples)
y_pred_score = np.clip(y_pred_score, 1, 10)

# Generate prediction probabilities
y_prob = np.random.dirichlet([2, 2, 2], size=n_samples)
# Adjust probabilities to be more consistent with predictions
for i in range(n_samples):
    y_prob[i, y_pred_class[i]] = max(y_prob[i, y_pred_class[i]], 0.6)
    y_prob[i] = y_prob[i] / y_prob[i].sum()  # Renormalize

print(f"\n🔢 Simulated evaluation data: {n_samples} samples")
print(f"True class distribution: {np.bincount(y_true_class)}")
print(f"Predicted class distribution: {np.bincount(y_pred_class)}")
print(f"True score range: {y_true_score.min():.1f} - {y_true_score.max():.1f}")
print(f"Predicted score range: {y_pred_score.min():.1f} - {y_pred_score.max():.1f}")

In [ ]:
# Evaluate multi-task performance
evaluation_results = metrics_evaluator.evaluate_multi_task(
    classification_true=y_true_class,
    classification_pred=y_pred_class,
    regression_true=y_true_score,
    regression_pred=y_pred_score,
    classification_prob=y_prob
)

print("\n📈 Classification Performance:")
class_metrics = evaluation_results['classification']
print(f"  Accuracy: {class_metrics['accuracy']:.3f}")
print(f"  Precision (macro): {class_metrics['precision_macro']:.3f}")
print(f"  Recall (macro): {class_metrics['recall_macro']:.3f}")
print(f"  F1 Score (macro): {class_metrics['f1_macro']:.3f}")

print("\n📊 Regression Performance:")
regr_metrics = evaluation_results['regression']
print(f"  MAE: {regr_metrics['mae']:.3f}")
print(f"  RMSE: {regr_metrics['rmse']:.3f}")
print(f"  R² Score: {regr_metrics['r2']:.3f}")

print("\n🎯 Target Compliance:")
target_f1 = config.get('evaluation.targets.clause_detection_f1', 0.82)
target_mae = config.get('evaluation.targets.risk_score_mae', 1.2)

f1_status = "✅ PASS" if class_metrics['f1_macro'] >= target_f1 else "❌ FAIL"
mae_status = "✅ PASS" if regr_metrics['mae'] <= target_mae else "❌ FAIL"

print(f"  F1 Score: {class_metrics['f1_macro']:.3f} vs target {target_f1:.3f} {f1_status}")
print(f"  MAE: {regr_metrics['mae']:.3f} vs target {target_mae:.3f} {mae_status}")

print(f"\n🏆 Overall Performance Score: {evaluation_results['overall_score']:.3f}")

In [ ]:
# Generate performance visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Classification confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_true_class, y_pred_class)
class_names = ['Low Risk', 'Medium Risk', 'High Risk']

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names, ax=axes[0,0])
axes[0,0].set_title('Classification Confusion Matrix')
axes[0,0].set_ylabel('True Label')
axes[0,0].set_xlabel('Predicted Label')

# Regression scatter plot
axes[0,1].scatter(y_true_score, y_pred_score, alpha=0.6, c='blue')
axes[0,1].plot([1, 10], [1, 10], 'r--', alpha=0.8)
axes[0,1].set_title('Regression: True vs Predicted Scores')
axes[0,1].set_xlabel('True Risk Score')
axes[0,1].set_ylabel('Predicted Risk Score')
axes[0,1].set_xlim(1, 10)
axes[0,1].set_ylim(1, 10)

# Regression residuals
residuals = y_pred_score - y_true_score
axes[1,0].scatter(y_true_score, residuals, alpha=0.6, c='green')
axes[1,0].axhline(y=0, color='red', linestyle='--')
axes[1,0].set_title('Regression Residuals')
axes[1,0].set_xlabel('True Risk Score')
axes[1,0].set_ylabel('Prediction Error')

# Performance metrics comparison
metrics_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
metrics_values = [
    class_metrics['accuracy'],
    class_metrics['precision_macro'],
    class_metrics['recall_macro'],
    class_metrics['f1_macro']
]
target_values = [0.85, 0.80, 0.80, target_f1]  # Example targets

x_pos = np.arange(len(metrics_names))
width = 0.35

axes[1,1].bar(x_pos - width/2, metrics_values, width, label='Actual', color='skyblue')
axes[1,1].bar(x_pos + width/2, target_values, width, label='Target', color='lightcoral')
axes[1,1].set_title('Classification Metrics vs Targets')
axes[1,1].set_xlabel('Metrics')
axes[1,1].set_ylabel('Score')
axes[1,1].set_xticks(x_pos)
axes[1,1].set_xticklabels(metrics_names, rotation=45)
axes[1,1].legend()
axes[1,1].set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 8. Interactive Risk Scoring {#interactive}

In [ ]:
# Interactive risk assessment function
def interactive_risk_assessment(text):
    """Assess risk for user-provided legal clause."""
    if not text.strip():
        return "Please provide a legal clause to analyze."
    
    # Clean the text
    cleaned_text = preprocessor.clean_legal_text(text)
    
    # Simple rule-based risk assessment (in production, this would use the trained model)
    risk_score = assess_clause_risk(text)
    
    if risk_score <= 3.5:
        risk_category = 'LOW RISK 🟢'
        recommendation = "This clause appears favorable to the employee."
    elif risk_score <= 7.0:
        risk_category = 'MEDIUM RISK 🟡'
        recommendation = "This clause has some concerning elements. Review carefully."
    else:
        risk_category = 'HIGH RISK 🔴'
        recommendation = "This clause may be heavily skewed against the employee. Consider negotiation."
    
    # Identify risk indicators
    text_lower = text.lower()
    high_risk_found = []
    protective_found = []
    
    high_risk_indicators = [
        'without cause', 'without notice', 'immediately', 'at will',
        'sole discretion', 'worldwide', 'unlimited', 'perpetual'
    ]
    
    protective_indicators = [
        'with notice', 'reasonable', 'fair', 'written notice',
        'mutual', 'subject to', 'limited'
    ]
    
    for indicator in high_risk_indicators:
        if indicator in text_lower:
            high_risk_found.append(indicator)
    
    for indicator in protective_indicators:
        if indicator in text_lower:
            protective_found.append(indicator)
    
    # Format results
    results = f"""
📄 LEGAL CLAUSE RISK ASSESSMENT
{'='*50}

Original Text:
{text}

Risk Score: {risk_score:.1f}/10
Risk Level: {risk_category}

Recommendation:
{recommendation}

Risk Indicators Found:
❌ High-risk terms: {', '.join(high_risk_found) if high_risk_found else 'None detected'}
✅ Protective terms: {', '.join(protective_found) if protective_found else 'None detected'}

Analysis Details:
• Text length: {len(text)} characters
• Word count: {len(text.split())} words
• Cleaned text: {cleaned_text[:100]}{'...' if len(cleaned_text) > 100 else ''}
    """
    
    return results

print("🎮 Interactive Legal Clause Risk Assessment")
print("You can now analyze any legal clause for risk factors!")
print("\nTry analyzing these example clauses:")

In [ ]:
# Example 1: High-risk clause
high_risk_clause = "The Company may terminate the Employee at any time, with or without cause, and with or without notice, at the sole discretion of management."

print("🔴 EXAMPLE 1 - High Risk Clause:")
print(interactive_risk_assessment(high_risk_clause))

In [ ]:
# Example 2: Low-risk clause
low_risk_clause = "Either party may terminate this agreement with 30 days written notice, subject to completion of current projects and fair transition arrangements."

print("🟢 EXAMPLE 2 - Low Risk Clause:")
print(interactive_risk_assessment(low_risk_clause))

In [ ]:
# Example 3: Medium-risk clause
medium_risk_clause = "Employee agrees not to disclose confidential information and not to compete with similar businesses for 12 months after termination."

print("🟡 EXAMPLE 3 - Medium Risk Clause:")
print(interactive_risk_assessment(medium_risk_clause))

In [ ]:
# Custom clause analysis
print("✏️ Analyze Your Own Clause:")
print("Replace the text below with your own legal clause to analyze:")

# User can modify this text
custom_clause = "Enter your legal clause here for risk assessment."

if custom_clause != "Enter your legal clause here for risk assessment.":
    print("\n🔍 CUSTOM CLAUSE ANALYSIS:")
    print(interactive_risk_assessment(custom_clause))
else:
    print("\n💡 Tip: Replace the custom_clause variable above with your own legal text to see the risk assessment!")
    print("\nFor example, try:")
    print('custom_clause = "Your legal clause text here"')

## Summary and Next Steps

🎉 **Congratulations!** You've explored the Legal Clause Risk Scorer system.

### What We've Covered:
1. ✅ **Data Loading**: CUAD and LEDGAR dataset integration
2. ✅ **Text Preprocessing**: Legal-specific cleaning and feature extraction
3. ✅ **Model Architecture**: Multi-task transformer for classification and regression
4. ✅ **Risk Assessment**: Automated scoring of legal clauses
5. ✅ **Performance Analysis**: Comprehensive evaluation metrics
6. ✅ **Interactive Demo**: Real-time clause analysis

### Key Features:
- **Multi-task Learning**: Simultaneous classification and risk scoring
- **Legal Domain Expertise**: Specialized preprocessing and evaluation
- **Production Ready**: MLflow tracking, checkpointing, comprehensive testing
- **Interpretable**: Attention weights and feature analysis

### Performance Targets:
- 🎯 **F1 Score**: ≥ 0.82 for clause detection
- 🎯 **MAE**: ≤ 1.2 for risk score prediction
- 🎯 **Recall**: ≥ 0.88 for unfavorable term detection

### Next Steps:
1. **Train the Model**: Run `python scripts/train.py` to train on full datasets
2. **Evaluate Performance**: Use `python scripts/evaluate.py` for comprehensive testing
3. **Deploy for Production**: Integrate with legal document review workflows
4. **Continuous Learning**: Update model with new legal clause patterns

### Usage Examples:
```bash
# Train the model
python scripts/train.py

# Evaluate on test data
python scripts/evaluate.py --model models/best_model.pth --report

# Analyze custom clauses
python scripts/evaluate.py --model models/best_model.pth --data my_clauses.csv
```

---
*The Legal Clause Risk Scorer helps identify potentially unfavorable terms in employment contracts and NDAs, focusing on asymmetric risk detection to protect employee interests.*